# Whole notebook in a nutshell

bash /scratch/iss_decoding/nf/workflow-decoding/local_run.sh \
    /scratch/tl10/decoding/decoding.yaml

In [1]:
! cat /scratch/tl10/decoding/decoding.yaml

codebook : /scratch/iss_decoding/data/decoding/in/NT_FLW mouse brain CNS codebook.xlsx
out_dir : ./out/
ome_tif : /scratch/iss_decoding/data/registration/out/demo_optflow_seg_optflow_reg_result_stack.tif
channel_map : "{'Cy5':'A','AF488':'G','Cy3':'C','Atto425':'T','AF750':'T'}"
rna_spot_size : [5]
anchor_ch_indexes : 1
prob_threshold : 0.9
trackpy_percentile : [99, 98]
trackpy_separation : 3
codebook_sep : ','

bf2raw_sif : '/scratch/iss_decoding/sifs/bf2raw-0.4.0.sif'
gmm_sif : '/scratch/iss_decoding/sifs/gmm_decode.sif'


# Or step by step ...

In [3]:
# import tifffile as tf
from dask_image.imread import imread
import matplotlib.pyplot as plt
use_gpu = True
if use_gpu:
    from cucim.skimage.morphology import white_tophat, disk
    import cupy as xp
    print("Using cucim")
else:
    from skimage.morphology import white_tophat, disk
    import numpy as xp
    print("Using skimage")
from tqdm import tqdm

Using cucim


In [8]:
root = "/scratch/iss_decoding/workshop_data/decoding/"
out_dir = "./out"

In [5]:
! mkdir {out_dir}

mkdir: cannot create directory ‘./out’: File exists


In [6]:
# Lazily load the image
registered_stack = imread(f"{root}/in/demo_optflow_seg_optflow_reg_result_stack.tif")

In [7]:
registered_stack.shape # (c, y, x), 35 channels of 4114 * 4369 images

(35, 4114, 4369)

In [8]:
anchor_ch = registered_stack[1]
anchor_ch
print("Anchor image shape: ", anchor_ch.shape)
print("Channel 1-4 are empty channels, not used for decoding")
coding_chs = registered_stack[
    [6,7,8,9,
     11,12,13,14,
     16,17,18,19,
     21,22,23,24,
     26,27,28,29,
     31,32,33,34]]
print("Coding image shape: ", coding_chs.shape)

Anchor image shape:  (4114, 4369)
Channel 1-5 are empty channels, not used for decoding
Coding image shape:  (24, 4114, 4369)


In [9]:
import napari

In [10]:
viewer = napari.Viewer()

In [11]:
viewer.add_image(registered_stack)

<Image layer 'registered_stack' at 0x7fdaf77fddf0>

In [22]:
spot_diam = 5 # manually count the spot diameter. Roughly 5 pixels.

In [23]:
selem=disk(spot_diam/2) # Create the kernel for white tophat filtering

In [24]:
selem # TODO: Napari vis.

array([[0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0]], dtype=uint8)

In [15]:
viewer.add_image(selem.get(), name="kernel")

<Image layer 'kernel' at 0x7fda1819d3d0>

In [16]:
enhancded = []
for ch in tqdm(coding_chs): # Run the white tophat filter on all coding channels
    enhancded.append(white_tophat(xp.array(ch), footprint=selem))

100%|██████████| 24/24 [00:02<00:00,  8.15it/s]


In [17]:
enhanced_stack = xp.array(enhancded)

In [25]:
viewer.add_image(enhanced_stack.get(), name="enhanced_stack")

<Image layer 'enhanced_stack [1]' at 0x7fdb77511e50>

In [26]:
enhancded_anchor = white_tophat(xp.array(anchor_ch), footprint=selem) # Run the white tophat filter on the anchor channel

In [27]:
viewer.add_image(enhancded_anchor.get(), name="enhanced_anchor_channel")

<Image layer 'enhanced_anchor_channel' at 0x7fdb76a86b50>

In [3]:
import trackpy as tp
import numpy as np

In [4]:
spots = tp.locate(np.array(enhancded_anchor.get()), spot_diam, separation=3, percentile=95) # Find the spots in the anchor channel

NameError: name 'enhancded_anchor' is not defined

In [ ]:
spots

In [35]:
viewer.add_points(spots[['y', 'x']], face_color="red", opacity=0.5, name="all_RNA_spots")

<Points layer 'all_RNA_spots' at 0x7fd9a86ea1c0>

In [36]:
spots["x_int"] = spots["x"].astype(int)
spots["y_int"] = spots["y"].astype(int)

In [37]:
spots.to_csv(f"{out_dir}/demo_optflow_seg_optflow_reg_result_stack_spots.csv",
            index=False)

In [38]:
enhanced_stack.shape

(24, 4114, 4369)

In [39]:
peak_profiles = enhanced_stack[:, spots["y_int"], spots["x_int"]]

In [40]:
peak_profiles.shape

(24, 69323)

In [41]:
reshaped_profiles = peak_profiles.reshape((4, 6, peak_profiles.shape[1])) # Reshape the peak profiles into a 4 barcodes * 6 cycles * 35 channels array

In [29]:
reshaped_profiles.shape

(4, 6, 32243)

In [43]:
first_profile = reshaped_profiles[:,:,1000]
first_profile

array([[ 0,  3,  0,  5,  0,  0],
       [ 0,  6, 60,  5,  1,  4],
       [ 0, 22,  4,  0, 31,  3],
       [ 0,  2,  0,  7,  2,  3]], dtype=uint16)

In [44]:
# Ont-hot encoding
first_profile.argmax(axis=0)

array([0, 2, 1, 3, 2, 1], dtype=int64)

In [45]:
np.save(
    f"{out_dir}/demo_optflow_seg_optflow_reg_result_stack_peak_profiles.npy",
    np.transpose(reshaped_profiles, (2, 0, 1)).astype(np.int32),
    allow_pickle=True
)

# Decoding - Codebook preprocessing

In [46]:
import bin.codebook_convert as cc
import bin.reading_data_functions as rdf
import bin.decode as dc
import pickle
import numpy as np

In [47]:
codebook_p = cc.main(f"{root}/in/NT_FLW mouse brain CNS codebook.xlsx", out_dir=f"{out_dir}/")

OrderedDict([('Cy5', 'A'), ('AF488', 'G'), ('Cy3', 'C'), ('Atto425', 'T'), ('AF750', 'T')])
  nCycles nChannel    DAPI Cy5 AF488 Cy3 Atto425
0       6        5  nuclei   A     G   C       T
       Gene Channel
0      Ache  ACTTGA
1     Acta2  GTATCC
2     Aldoc  CCCATT
3      Bdnf  TTCGTC
4     Calca  AAACGT
5      Chat  CATAGG
6       Dcn  ATTCAG
7       Fev  TGCGAT
8       Fos  CTGCAC
9      Gad1  GTTTGA
10     Gad2  GATTCA
11     Gfap  ACAGCG
12     Gja1  CACTAT
13    Itgam  CTCGCG
14    Kcnj8  AGAGAT
15   Laptm5  CGTTAT
16     Map2  TCTTTG
17      Mbp  GCTCAC
18     Ndnf  CTACGT
19     Nefh  GCAATT
20  Neurod1  ATCCGA
21      Npy  ACCTAT
22    Npy2r  GTCCAC
23    Ntrk2  GGCGCA
24      Oxt  AGTGCT
25    P2rx3  CGAATT
26     Pcp4  CCTTCG
27     Pdyn  CGGACA
28     Plp1  ACGGTC
29    Ptprc  CAGCTC
30    Pvalb  ATGCCC
31   Rbfox3  GAGTGA
32  Slc17a6  GCGTCG
33  Slc17a7  TTTACA
34   Slc6a1  TACAGC
35   Slc6a3  TGAACC
36   Slc6a4  CTTCTG
37   Slc6a5  CCGTGG
38     Sncg  CAAACG
39     Spp

In [48]:
barcodes_01, K, R, C, gene_names, channels_info = rdf.read_taglist_and_channel_info(
        f"{out_dir}/",
        taglist_name="taglist.csv",
        channel_info_name="channel_info.csv",
)

In [49]:
np.save(f"{out_dir}/barcodes_01.npy", barcodes_01)
np.save(f"{out_dir}/gene_names.npy", gene_names)
channels_info["K"] = K
channels_info["R"] = R
channels_info["C"] = C
print(channels_info)
with open(f"{out_dir}/channel_info.pickle", "wb") as fp:
    pickle.dump(channels_info, fp)

{'barcodes_AGCT': array(['ACTTGA', 'GTATCC', 'CCCATT', 'TTCGTC', 'AAACGT', 'CATAGG',
       'ATTCAG', 'TGCGAT', 'CTGCAC', 'GTTTGA', 'GATTCA', 'ACAGCG',
       'CACTAT', 'CTCGCG', 'AGAGAT', 'CGTTAT', 'TCTTTG', 'GCTCAC',
       'CTACGT', 'GCAATT', 'ATCCGA', 'ACCTAT', 'GTCCAC', 'GGCGCA',
       'AGTGCT', 'CGAATT', 'CCTTCG', 'CGGACA', 'ACGGTC', 'CAGCTC',
       'ATGCCC', 'GAGTGA', 'GCGTCG', 'TTTACA', 'TACAGC', 'TGAACC',
       'CTTCTG', 'CCGTGG', 'CAAACG', 'CGCGGA', 'GGTCTG', 'GTGGTC',
       'TTATAC', 'AATATG', 'CCATAA', 'GGAAGC', 'TGTCGA', 'TATGAT'],
      dtype=object), 'coding_chs': [False, True, True, True, True], 'channel_base': ['nuclei', 'A', 'G', 'C', 'T'], 'channel_names': ['DAPI', 'Cy5', 'AF488', 'Cy3', 'Atto425'], 'K': 48, 'R': 6, 'C': 4}


In [ ]:
dc.decode(
    "EMBL_training_mouse_brain",
    f"{out_dir}/demo_optflow_seg_optflow_reg_result_stack_peak_profiles.npy",
    f"{out_dir}/demo_optflow_seg_optflow_reg_result_stack_spots.csv",
    f"{out_dir}/barcodes_01.npy", f"{out_dir}/gene_names.npy", f"{out_dir}/channel_info.pickle",
    out_dir=f"{out_dir}"
)

(69323, 4, 6)                        x          mass      size       ecc       signal  \
y                                                                         
1.811746      153.978805   3244.690741  1.108477  0.238506   647.940118   
1.663028      158.088624   1557.239474  1.172487  0.505696   306.894381   
2.325531      268.008340    579.637405  1.226007  0.438635   101.362473   
1.970572      293.804188    551.099970  1.371346  0.713368   102.298127   
2.093709      311.700856   1549.286419  1.116964  0.305634   297.381902   
...                  ...           ...       ...       ...          ...   
4110.521897  1805.929931    516.324845  1.361884  0.449990   155.162555   
4111.186558  1818.722526   2078.866356  1.158376  0.354676   374.105498   
4110.787751  1988.926367    667.121017  1.173277  0.416416   113.370028   
4110.975786  3309.609737   1210.735758  1.157263  0.393646   218.631058   
4111.084381  3455.877532  11476.571112  1.159388  0.417885  2162.919238   

          

In [38]:
root = "/scratch/iss_decoding/workshop_data/decoding/"
out_dir = "./out"

In [39]:
 ! ls {out_dir}

EMBL_training_mouse_brain_decode_out_parameters.pickle
EMBL_training_mouse_brain_decoded_df.tsv
barcodes_01.npy
channel_info.csv
channel_info.pickle
demo_optflow_seg_optflow_reg_result_stack_peak_profiles.npy
demo_optflow_seg_optflow_reg_result_stack_spots.csv
gene_names.npy
taglist.csv


# Visual QC of the decoded data

In [5]:
import pandas as pd
import napari

In [6]:
viewer = napari.Viewer()

In [9]:
decoded_peaks = pd.read_csv(f"{out_dir}/EMBL_training_mouse_brain_decoded_df.tsv", sep="\t")

FileNotFoundError: [Errno 2] No such file or directory: './out/EMBL_training_mouse_brain_decoded_df.tsv'

In [42]:
decoded_peaks

,Name,Code,Probability,y_int,x_int,index_code,axis,R0_C0,R1_C0,R2_C0,...,R2_C2,R3_C2,R4_C2,R5_C2,R0_C3,R1_C3,R2_C3,R3_C3,R4_C3,R5_C3
0,background,0000,0.964294,1,153,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fos,CTGCAC,0.158820,2,829,132141.0,1,0,0,5,...,2,3,0,0,0,0,0,0,0,0
2,Bdnf,TTCGTC,0.269279,1,969,331231.0,1,0,0,0,...,2,0,0,9,898,5,0,0,0,0
3,Trh,CCATAA,0.524485,1,1044,114344.0,1,0,0,45,...,4,0,0,0,0,0,0,0,0,0
4,Pdyn,CGGACA,0.797838,2,2087,122414.0,1,0,0,4,...,0,0,4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32238,background,0000,0.964294,4110,1043,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32239,background,0000,0.964294,4110,1162,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32240,background,0000,0.964294,4110,1583,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32241,background,0000,0.964294,4110,1788,NaN,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
decoded_group = decoded_peaks[decoded_peaks.Probability > 0.9]

In [44]:
decoded_peaks.Name.unique()

array(['background', 'Fos', 'Bdnf', 'Trh', 'Pdyn', 'Gfap', 'Itgam',
       'Acta2', 'Slc17a7', 'Ntrk2', 'Npy2r', 'Slc6a3', 'Laptm5', 'Fev',
       'Neurod1', 'infeasible', 'Chat', 'Oxt', 'Npy', 'Pvalb', 'Calca',
       'Sst', 'Gad1', 'Gja1', 'Mbp', 'Gad2', 'Kcnj8', 'P2rx3', 'Plp1',
       'Sncg', 'Ptprc', 'Tubb3', 'Ache', 'Slc6a4', 'Ndnf', 'Th', 'Slc6a5',
       'Vip', 'Slc6a1', 'Syn1', 'Rbfox3', 'Dcn', 'Spp1', 'Trpv1', 'Tph1',
       'Nefh', 'Map2', 'Slc17a6', 'Pcp4', 'Aldoc'], dtype=object)

In [45]:
decoded_group = decoded_peaks.groupby("Name")

In [46]:
spot_layers = {}
for n, grp in decoded_group:
    # print(n, grp)
    spot_layers[n] = viewer.add_points(grp[['y_int', 'x_int']], name=n)

In [47]:
for n in spot_layers:
    spot_layers[n].visible=False

In [48]:
spot_layers['Vip'].face_color= 'red'
spot_layers['Vip'].visible= True
spot_layers['Vip'].size= 100

More in https://napari.org/stable/howtos/layers/points.html